<a href="https://colab.research.google.com/github/HaoYamado/notebooks/blob/master/CNN_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network with PyTorch

### Basic Convoluational Neural Network (CNN)
*   A basic CNN just requires 2 additional layers!
    *    Convolutional and pooling layers before our feedforward neural network

**Fully Connected (FC) Layer**
A layer with an affine function & non-linear function is called a Fully Connected (FC) Layer
![alt text](https://www.deeplearningwizard.com/deep_learning/practical_pytorch/images/cnn1.png)

##  One Convolutional Layer: High Level View
![alt text](https://www.deeplearningwizard.com/deep_learning/practical_pytorch/images/cnn6-1.png)

##  One Convolutional Layer: High Level View Summary
![alt text](https://www.deeplearningwizard.com/deep_learning/practical_pytorch/images/cnn2.png)

*   As the kernel is sliding/convolving across the image ---> 2 operations
done per patch
    *  a. Element-wise multiplication
    *  b. Summation
*   More kernels = more feature map channels
    * Can capture more inforamtion about the input

##  Multiple Convolutional Layers: High Level View
![alt text](https://www.deeplearningwizard.com/deep_learning/practical_pytorch/images/cnn7-3.png)

##  Pooling Layer: High Level View
*   2 Common Types
    *  Max Pooling
    *  Average Pooling
![alt text](https://www.deeplearningwizard.com/deep_learning/practical_pytorch/images/cnn8n2.png)

##  Multiple Pooling Layers: High Level View
![alt text](https://www.deeplearningwizard.com/deep_learning/practical_pytorch/images/cnn7-3.png)

## Padding
![alt text](https://www.deeplearningwizard.com/deep_learning/practical_pytorch/images/cnn9-4.png)

##  Padding Summary
*   Valid Padding (No Padding)
   *   Output size < input size
*   Same Padding (Zero Padding)
   *   Output size = input size

# Building a Covolutional Neural Network with PyTorch
### Model A:
*   2 Convolutional Layers
    * Same Padding( same output size)
*   2 Max Pooling Layers
*   1 Fully Connected Layer
![alt text](https://www.deeplearningwizard.com/deep_learning/practical_pytorch/images/cnn10-1.png)

**Step 1: Loading MNIST Train Dataset**





In [0]:
import torch 
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

train_dataset = dsets.MNIST(root='./data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset =dsets.MNIST(root='./data',
                          train=False,
                          transform=transforms.ToTensor())


In [15]:
print(train_dataset.train_data.size())

torch.Size([60000, 28, 28])


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [16]:
print(train_dataset.train_labels.size())

torch.Size([60000])


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [17]:
print(test_dataset.test_data.size())

torch.Size([10000, 28, 28])


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


In [18]:
print(test_dataset.test_labels.size())

torch.Size([10000])


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


### Step 2: Make Dataset Iterable

In [0]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters /(len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

### Create Model Class
![alt text](https://www.deeplearningwizard.com/deep_learning/practical_pytorch/images/cnn10-1.png)
![alt text](https://sun9-47.userapi.com/c858224/v858224001/d8fad/zYupcCK2i_A.jpg)
![alt text](https://www.deeplearningwizard.com/deep_learning/practical_pytorch/images/cnn10-2n.png)

In [0]:
class CNNModel(nn.Module):
  def __init__(self):
    super(CNNModel, self).__init__()
    # Conv 1
    self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
    self.relu1 = nn.ReLU()

    # Max Pool 1
    self.maxpool1 = nn.MaxPool2d(kernel_size=2)

    # Conv2 
    self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
    self.relu2 = nn.ReLU()

    # Max Pool 2
    self.maxpool2 = nn.MaxPool2d(kernel_size=2)

    # Fully Connected 1(readout)
    self.fc1 = nn.Linear(32 * 7 * 7, 10)

  def forward(self, x):
    # Conv 1
    out = self.cnn1(x)
    out = self.relu1(out)

    # Max Pool 1
    out = self.maxpool1(out)

    # Conv 2 
    out = self.cnn2(out)
    out = self.relu2(out)

    # Max Pool 2
    out = self.maxpool2(out)

    # Resize
    # Original size: (00, 32, 7, 7)
    # out.size(100, 32*7*7)
    out = out.view(out.size(0), -1)

    # Linear function (readout)
    out = self.fc1(out)
    return out


### Step 4: Instantiate Model Class

In [0]:
# our model
model = CNNModel()

### Step 5: Instantiate Loss Class

In [0]:
criterion = nn.CrossEntropyLoss()

### Step 6: Instantiate Optimizer Class

In [0]:
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### Paramters In-Depth

In [24]:
print(model.parameters())
print(len(list(model.parameters())))

# Conv 1: 16 kernels
print(list(model.parameters())[0].size())

# Conv 1 Bias: 16 Kernels
print(list(model.parameters())[1].size())

# Conv 2: 32 kernels with depth = 16
print(list(model.parameters())[2].size())

# Conv 2 Bias: 32 kernels with depth = 16
print(list(model.parameters())[3].size())

# Fully Connected Layer 1
print(list(model.parameters())[4].size())

# Fully Connected Layer 1 Bias
print(list(model.parameters())[5].size())

<generator object Module.parameters at 0x7fe59e6f0fc0>
6
torch.Size([16, 1, 5, 5])
torch.Size([16])
torch.Size([32, 16, 5, 5])
torch.Size([32])
torch.Size([10, 1568])
torch.Size([10])


### Step 7: Train Model



*   Process
    *  a. Convert inputs to tensors with gradient accumulation abilities
      *   CNN Input: (1, 28, 28)
      *   Feedforward NN input: (1, 28*28)
    *  b. Clear gradient buffers
    *  c. Get output given inputs
    *  d. Get loss
    *  e. Get gradients w.r.t. parameters
    *  f. Update parameters using gradients
      *   parameters = parameters - learning_rate * parameters_gradients  
    *  g. REPEAT



In [25]:
iter = 0
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    # load images
    images = images.requires_grad_()
    # Clear gradients w.r.t. parameters
    optimizer.zero_grad()
    # Forward pass to get output/logits
    outputs = model(images)
    # Calculate Loss: softmax ---> cross entropy loss
    loss = criterion(outputs, labels)
    # Getting grdient w.r.t. parameters
    loss.backward()
    # Updatin parameters
    optimizer.step()

    iter += 1
    if iter % 500 == 0:
      # Calculate Accuracy
      correct = 0
      total = 0
      # iterate through test dataset
      for images, labels in test_loader:
        # load images
        images = images.requires_grad_()
        # Forward pass only to get logits/outputs
        outputs = model(images)
        # Get predictions form maximum value
        _, predicted = torch.max(outputs.data, 1)
        # Total number of labels
        total += labels.size(0)
        # Total correct predictions
        correct += (predicted == labels).sum()
      accuracy = 100 * correct / total
      print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 0.5138514041900635. Accuracy: 89
Iteration: 1000. Loss: 0.2485535442829132. Accuracy: 92
Iteration: 1500. Loss: 0.1941078007221222. Accuracy: 94
Iteration: 2000. Loss: 0.16878528892993927. Accuracy: 95
Iteration: 2500. Loss: 0.1298305243253708. Accuracy: 96
Iteration: 3000. Loss: 0.10238640755414963. Accuracy: 96


# Model B: 



*   2 Convolutional Layers
    *   Same Padding(same outputs size)
*   2 Average Pooling Layers
*   1 Fully Connected Layer

![alt text](https://www.deeplearningwizard.com/deep_learning/practical_pytorch/images/cnn10-4.png)





In [0]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

"""
STEP 1: LOADING DATASET
"""
train_dataset = dsets.MNIST(root='./data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())

"""
STEP 2: MAKING DATASET ITERABLE
"""
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

"""
STEP 3: CREATE MODEL CLASS
"""

class CNNModel(nn.Module):
  def __init__(self):
    super(CNNModel, self).__init__()

    # Coinvolutional 1
    self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
    self.relu1 = nn.ReLU()

    # Average pool 
    self.avgpool1 = nn.AvgPool2d(kernel_size=2)

    # Convolutional 2
    self.cnn2 = nn.Conv2d(in_channels=26, out_channels=32, kernel_size=5, stride=1, padding=2)
    self.relu = nn.ReLU()

    # Average pool 2
    self.avgpool2 = nn.AvgPool2d(kernel_size=2)
    #  Fully connected 1 (readout)
    self.fc1 = nn.Linear(32 * 7 * 7, 10)

  def forward(self, x):
    # Convolutional 1
    out = self.cnn1(x)
    out = self.relu1(out)

    # Average pool 1
    out = self.avgpool1(out)
    
    # Convolutional 2
    out = self.cnn2(out)
    out = self.relu2(out)

    # Average pool 2
    out = self.avgpool2(out)

    
    # Resize
    # Original size: (100, 32, 7, 7)
    # out.Size(0): 100
    # New out size: (100 , 32*7*7)
    out = out.view(out.size(0), -1)
    
    # Linear function (readout)
    out = self.fc1(out)
    return out

"""
STEP 4: INSTANTIATE MODEL CLASS
"""
model = CNNModel()

"""
STEP 5: INSTANTIALE LOSS CLASS
"""
criterion = nn.CrossEntropyLoss()

"""
STEP 6: INSTANTIATE OPTIMIZER CLASS
"""
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
